# Adult brain measure estimation from BLS data - BLS-38
As BLS is not publicly available due to ethical considerations, no data is included in this repository. The code is provided as a reference for the analysis of the BLS data. 

Since not all participants had a longitudinal scan yet, BLS-26 and BLS-38 were preprocessed in two different repositories.

In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Data preparation
This whole notebook can be run for cortical thickness (CT) or surface area (SA). Both measures were used in the original publication. Adjust the variable `brain_measure` to `CT` or `SA` accordingly. 

In [16]:
import os
from os.path import join
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

from abagen import fetch_desikan_killiany
from neuroCombat import neuroCombat

import matplotlib.pyplot as plt
import seaborn as sns

# import custom code
import sys
sys.path.append('code')
from utils import reorder_vars
from preprocessing import load_freesurfer_aparc, load_freesurfer_aseg, calculate_scanner_difference
from euler import euler_hemi_combination, get_euler_outliers

bls_data_dir = 'data/BLS'
bls_freesurfer_outputs = join(bls_data_dir, 'freesurfer')  # where all freesurfer outputs are stored
qc_dir_bls38 = join(bls_data_dir, 'euler_BLS-38')
bls_out = join(bls_data_dir, 'derivatives')  # where the adapted data will be stored
os.makedirs(bls_out, exist_ok=True)


# adjust brain measurement for which the code should be run
brain_measure = 'CT'  # CT or SA

# Load MRI related data

In [17]:
# brain_measure
if brain_measure == 'CT':
    mri_data_lh = load_freesurfer_aparc(file = join(bls_freesurfer_outputs, 'BLS-38_thickness_fs7.3.2_lh.txt'))
    mri_data_rh = load_freesurfer_aparc(file = join(bls_freesurfer_outputs, 'BLS-38_thickness_fs7.3.2_rh.txt'))
    mri_data = mri_data_lh.merge(mri_data_rh, on='participant')
    #mri_data.drop(columns=['lh_MeanThickness_thickness', 'rh_MeanThickness_thickness'], inplace=True)
    mri_data.rename(columns={'lh_MeanThickness_thickness':'L_mean_CT2', 'rh_MeanThickness_thickness':'R_mean_CT2'}, inplace=True)
    bls_idps_idx = mri_data.filter(regex='_thickness').columns.to_list()
    
elif brain_measure == 'SA':
    mri_data_lh = load_freesurfer_aparc(file = join(bls_freesurfer_outputs, 'BLS-38_surfaceArea_fs7.3.2_lh.txt'))
    mri_data_rh = load_freesurfer_aparc(file = join(bls_freesurfer_outputs, 'BLS-38_surfaceArea_fs7.3.2_rh.txt'))
    mri_data = mri_data_lh.merge(mri_data_rh, on='participant')
    mri_data.drop(columns=['lh_WhiteSurfArea_area', 'rh_WhiteSurfArea_area'], inplace=True)
    bls_idps_idx = mri_data.filter(regex='_area').columns.to_list()
    
else:
    raise ValueError('brain_measure has to be either CT or SA')


# aseg
aseg = load_freesurfer_aseg(file = join(bls_freesurfer_outputs, 'BLS-38_aseg_fs7.3.2.txt'))
aseg = aseg[['participant', 'EstimatedTotalIntraCranialVol','TotalGrayVol','SubCortGrayVol','CerebralWhiteMatterVol']]    

# meta
meta = pd.read_csv(join(bls_data_dir, 'BLS-38_meta_cleaned.csv'))
meta = meta[['participant', 'blsgroup','phenotype','age', 'sex','sex_code']]

# add additional information from BLS-26 meta
meta_26 = pd.read_csv(join(bls_data_dir, 'BLS-26_meta_cleaned.csv'))
meta_26 = meta_26[['participant', 'GA', 'BW']]

# merge
bls_38 = meta.merge(meta_26, on='participant')
bls_38 = bls_38.merge(mri_data, on='participant')
bls_38 = bls_38.merge(aseg, on='participant')

# sort data
bls_38.sort_values(by='participant', inplace=True)
print(f'{bls_38.shape[0]} participants in BLS had a longitundinal scan already and are included in the analysis')

110 participants in BLS had a longitundinal scan already and are included in the analysis


In [18]:
# access DK atlas and save idp_labels
atlas = fetch_desikan_killiany(surface=True)
atlas = pd.read_csv(atlas['info'])
atlas = atlas[(atlas['structure'] == 'cortex') & (atlas['hemisphere'] == 'L')]
atlas_labels_l = ['L_'+label for label in atlas['label']]
atlas_labels_r = ['R_'+label for label in atlas['label']]

desikan_idps = atlas_labels_l + atlas_labels_r
print(f"Number of IDPs: {len(bls_idps_idx)}")

Number of IDPs: 68


In [19]:
# rename columns
bls_38 = bls_38.rename(columns=dict(
        SubCortGrayVol="sGMV",
        TotalGrayVol="GMV",
        EstimatedTotalIntraCranialVol="eTIV",
        CerebralWhiteMatterVol="WMV"
))
bls_38 = bls_38.rename(columns=dict(zip(bls_idps_idx, desikan_idps)))

## Quality control using Euler Index
Freesurfer's Euler number is a measure of the topological complexity of the reconstructed cortical surface. The Euler number is expected to be 2 for a closed surface. Freesurfer's Euler number has previously been used to identify subjects with poor quality reconstructions. Some informative publications are listed below:
- [Rosen 2018](https://doi.org/10.1016/j.neuroimage.2017.12.059) 
- [Bethlehem 2022](https://doi.org/10.1038/s41586-022-04554-y)

The Euler Index can be extracted with *mris_euler_index* from the recon-all output.

In [20]:
# combine Euler Indices across hemispheres
euler_hemi_combination(euler_file=os.path.join(qc_dir_bls38, 'QC_output_BLS-38.txt'), outdir=qc_dir_bls38, dataset_name='BLS-38')

# calculate outliers based on Euler Indices
outliers = get_euler_outliers(os.path.join(qc_dir_bls38, 'QC_output_combined_BLS-38.txt'), outdir=qc_dir_bls38, threshold=4.5)

# adapt excluded subjects after manual QC
outliers.append('BEST-BN-009')  # this subject does not have information for gestational age and was thus excluded

# get outlier complete df
outlier_data = bls_38[bls_38['participant'].isin(outliers)]
outlier_data['blsgroup']

with open(join(bls_data_dir, f'BLS-38_{brain_measure}_excluded_subjects.txt'), 'w') as f:
    for item in outliers:
        f.write("%s\n" % item)

Median of Euler indices: -18.0
Median absolute deviation (MAD): 8.0
Euler index histogram saved to: data/BLS/euler_BLS-38/euler_histogram.png


In [21]:
# actually exclude the outliers
print("Data shape before exclusion:", bls_38.shape)
bls_38 = bls_38[~bls_38['participant'].isin(outliers)]

print("New size of data:", bls_38.shape)

Data shape before exclusion: (110, 82)
New size of data: (105, 82)


In [22]:
# save data before hemi averaging
# bls_38.to_csv(join(bls_out, f'BLS-38_{brain_measure}_raw.csv'), index=False)

## Average hemispheres

In [23]:
regions = [r[2:] for r in desikan_idps[:34]]

# Combine L_ and R_ values
for region in regions:
    bls_38[f'{brain_measure}_{region}'] = bls_38[[f'L_{region}', f'R_{region}']].mean(axis=1)
    # drop L_ and R_ columns
    bls_38 = bls_38.drop(columns=[f'L_{region}', f'R_{region}'])
    
if brain_measure == 'CT':
    bls_38['meanCT2'] = (bls_38['L_mean_CT2'] + bls_38['R_mean_CT2']) / 2
    
# create region list bilateral    
desikan_idps_bilateral = [f'{brain_measure}_{region}' for region in regions]
ctv_columns = ['GMV', 'WMV', 'sGMV', 'eTIV']

In [24]:
# make sure data is ordered correctly
bls_38 = bls_38.sort_values(by='participant')

# Adapt df for BrainChart framework
BrainChart needs a certain format of the data. We will adapt the data accordingly. More information can be found [here](https://brainchart.shinyapps.io/brainchart/).

In [25]:
# age
bls_38.rename(columns={'age': 'Age'}, inplace=True)
bls_38['age_days'] = (bls_38.Age * 365.245) + 280

# sex
bls_38['sex'] = bls_38['sex'].map({'female':'Female', 'male':'Male'})

# dx
bls_38['dx'] = bls_38['phenotype'].map({'term':'CN', 'preterm':'preterm'})

# other columns
bls_38['study'] = 'BLS-38'
bls_38['fs_version'] = 'Custom'
bls_38['country'] = 'Multisite'
bls_38['run'] = 1
bls_38['session'] = 1

# drop columns
bls_38.drop(columns=['blsgroup', 'phenotype', 'sex_code'], inplace=True)

# reshape and rename last things
bls_38_final = reorder_vars(['participant', 'Age', 'age_days', 'sex', 'study', 'fs_version','country', 'run', 
                            'session', 'dx'], bls_38, desikan_idps_bilateral)

In [26]:
# save
bls_38_final.to_csv(join(bls_out, f'BLS-38_{brain_measure}_preprocessed.csv'), index=False)

# Stats
Summary stats shown in Supp Table S1.

In [27]:
print("Overall number of unique subjects with longitudinal data: n =", len(bls_38_final))

bls_38_final_pt = bls_38_final[bls_38_final['dx'] == 'preterm']
bls_38_final_cn = bls_38_final[bls_38_final['dx'] == 'CN']

print('Preterm stats: n = ', len(bls_38_final_pt))
print(bls_38_final_pt['sex'].value_counts())
display(bls_38_final_pt[['Age', 'GA', 'BW']].describe().round(2))


Overall number of unique subjects with longitudinal data: n = 105
Preterm stats: n =  52
Male      30
Female    22
Name: sex, dtype: int64


,Age,GA,BW
count,52.00,52.00,52.00
mean,38.10,30.10,1318.56
std,0.46,1.89,335.99
min,37.36,25.00,730.00
25%,37.74,29.00,990.00
50%,38.19,30.00,1345.00
75%,38.41,31.00,1512.50
max,39.07,35.00,2070.00


In [28]:
print('Full-term stats: n = ', len(bls_38_final_cn))
print(bls_38_final_cn['sex'].value_counts())
display(bls_38_final_cn[['Age', 'GA', 'BW']].describe().round(2))

Full-term stats: n =  53
Male      30
Female    23
Name: sex, dtype: int64


,Age,GA,BW
count,53.00,53.00,53.00
mean,38.21,39.89,3474.89
std,0.40,1.09,419.71
min,37.41,37.00,2600.00
25%,37.93,39.00,3200.00
50%,38.22,40.00,3450.00
75%,38.55,41.00,3730.00
max,38.91,42.00,4670.00
